In [0]:
import pandas as pd

dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

if catalog == "" and schema == "":
    df = spark.sql("SELECT value FROM dbacademy.ops.meta WHERE key = 'schema_name'")

    catalog = "dbacademy"
    schema = df.first()["value"]

files_to_download = [
    {"file_name": "aeso-january-2024-demand.csv", 
     "download_url": 
         "https://raw.githubusercontent.com/david-hurley/databricks-aibi-day/refs/heads/main/data/aeso-january-2024-demand.csv", 
         "table_name": "aeso_january_2024_demand"
    },
    {"file_name": "aeso-january-2024-supply.csv", 
     "download_url": 
         "https://raw.githubusercontent.com/david-hurley/databricks-aibi-day/refs/heads/main/data/aeso-january-2024-supply.csv", 
         "table_name": "aeso_january_2024_supply"
    },
    {"file_name": "calgary-january-2024-temperature.csv", 
     "download_url": 
         "https://raw.githubusercontent.com/david-hurley/databricks-aibi-day/refs/heads/main/data/calgary-january-2024-temperature.csv", 
         "table_name": "calgary_january_2024_temperature"
    }
]

for file in files_to_download:
    file_name = file["file_name"]
    download_url = file["download_url"]
    table_name = file["table_name"]
    
    df = pd.read_csv(download_url)

    spark_df = spark.createDataFrame(df)

    spark_df = spark_df.withColumn("datetime_MST", spark_df["datetime_MST"].cast("timestamp"))
    
    spark_df.write.mode("overwrite").saveAsTable("{catalog}.{schema}.{table_name}".format(catalog=catalog, schema=schema, table_name=table_name))

In [0]:
print("Catalog: {catalog}".format(catalog=catalog))
print("Schema: {schema}".format(schema=schema))